In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
import torch
torch.cuda.is_available()

True

# 10.7 文本情感分类：使用循环神经网络

In [7]:
import time
import collections
import os
import random
import tarfile
import torch
from torch import nn
import torchtext
# from torchtext.vocab import vocab as Vocab
import torchtext.vocab as Vocab
import torch.utils.data as Data

from nltk.tokenize import word_tokenize, RegexpTokenizer
import nltk
nltk.download('punkt')

import sys
sys.path.append("..")

# import d2lzh_pytorch as d2l

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__, device)
# print(torch.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
1.9.0+cu102 cuda


## 10.7.1 文本情感分类数据
### 10.7.1.1 读取数据

In [9]:
BASE = os.getcwd()
os.listdir(BASE)

['.config', 'drive', 'sample_data']

In [10]:
# download data, this data will be saved in '/content' folder 
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar xfz aclImdb_v1.tar.gz

--2021-09-02 06:47:21--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  30.9MB/s    in 2.6s    

2021-09-02 06:47:24 (30.9 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [11]:
os.listdir(BASE)

['.config', 'aclImdb', 'drive', 'aclImdb_v1.tar.gz', 'sample_data']

In [33]:
# DATA_ROOT = os.path.join(datasets_root_path, 'Dive-into-DL-PyTorch', "data")
# DATA_ROOT = datasets_root_path
# os.listdir(DATA_ROOT)

In [34]:
# fname = os.path.join(DATA_ROOT, "aclImdb_v1.tar.gz")
# if not os.path.exists(os.path.join(DATA_ROOT, "aclImdb")):
#     print("从压缩包解压...")
#     with tarfile.open(fname, 'r') as f:
#         f.extractall(DATA_ROOT)

# import requests

# url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
# with open(filename, "wb") as f:
#     r = requests.get(url)
#     f.write(r.content)

In [12]:
from tqdm import tqdm
def read_imdb(folder=None, data_root=None):
    data = []
    for label in ['pos', 'neg']:
        folder_name = os.path.join(data_root, folder, label)
        for file in tqdm(os.listdir(folder_name)):
            with open(os.path.join(folder_name, file), 'rb') as f:
                review = f.read().decode('utf-8').replace('\n', '').lower()
                data.append([review, 1 if label == 'pos' else 0])
    random.shuffle(data)
    return data

data_root = os.path.join(BASE, "aclImdb")
train_data, test_data = read_imdb('train', data_root), read_imdb('test', data_root)

100%|██████████| 12500/12500 [00:00<00:00, 32086.64it/s]


In [13]:
type(train_data), type(test_data), len(train_data), len(test_data)

(list, list, 25000, 25000)

In [14]:
train_data[0], test_data[0]

(['i really enjoyed this movie. the script is fresh and unpredictable and the acting is outstanding.it is a down-to-earth movie with characters one cares about. it brought tears into my eyes a few times but left me with a great feeling afterwards.',
  1],
 ['not near as well made as the "guinea pig" flicks it was inspired by ("flowers of flesh and blood" or "devil\'s experiment") and not conveying any real philosophy, this video feature, which is barely feature length, adds hardcore sex (with mosaic censoring) to its inspiration. the special make-up effects, which include stomach slitting and disembowelment, are pretty good, if overlit. the amateur feel of the production is a distraction. it all looks cheap and lazy. the lighting is harsh and the sound and editing are sloppy. the simple story involves a porno actress who ends up starring in a real snuff movie. just when she gets tired of being abused, the real abuse begins. in the film\'s hero scene, an actor cuts the woman\'s stomach 

### 10.7.1.2 预处理数据

In [15]:
def get_tokenized_imdb(data):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    """
    data: list of [string, label]
    """
    def tokenizer(text):
        return [tok for tok in text.lower().split(' ')]
    return [tokenizer(review) for review, _ in data]

# It turns out this tokenizer won't pass words like "don't", "won't", etc.
# def get_tokenized_imdb(data, tokenizer = RegexpTokenizer(r'\w+')):
#     """
#     data: list of [string, label]
#     """
#     return [tokenizer.tokenize(review) for review, _ in data]

In [16]:
tokenized_data = get_tokenized_imdb(train_data)
len(tokenized_data)

25000

In [17]:
# train_data[0][0]
# tokenized_data[0]

In [18]:
counter = collections.Counter([tk for st in tokenized_data for tk in st])

In [19]:
counter["the"], counter["a"]

(336749, 163123)

In [20]:
sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)

In [21]:
sorted_by_freq_tuples[0], sorted_by_freq_tuples[-1]

(('the', 336749), ('defilers', 1))

In [22]:
ordered_dict = collections.OrderedDict(sorted_by_freq_tuples)

In [23]:
# ordered_dict

In [24]:
v1 = Vocab.vocab(ordered_dict, min_freq=5)
# v1 = Vocab.vocab(counter, min_freq=5)

In [25]:
# v1.get_itos()

In [26]:
def get_vocab_imdb(data):  # 本函数已保存在d2lzh_pytorch包中方便以后使用
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: x[1], reverse=True)
    ordered_dict = collections.OrderedDict(sorted_by_freq_tuples)
    return Vocab.vocab(ordered_dict, min_freq=3)

vocab_imdb = get_vocab_imdb(train_data)

In [27]:
'# words in vocab_imdb:', len(vocab_imdb)

('# words in vocab_imdb:', 37893)

In [28]:
vocab_imdb.get_itos()[0]

'the'

In [29]:
vocab_imdb["a"]

2

In [30]:
unk_token = "<unk>"
vocab_imdb.insert_token(unk_token, 0)

In [31]:
vocab_imdb.get_itos()[:10]

['<unk>', 'the', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in']

In [32]:
vocab_imdb.set_default_index(vocab_imdb[unk_token])

In [33]:
def preprocess_imdb(data, vocab):  # 本函数已保存在d2lzh_torch包中方便以后使用
    max_l = 500  # 将每条评论通过截断或者补0，使得长度变成500

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    features = torch.tensor([pad([vocab[word] for word in words]) for words in tokenized_data])
    labels = torch.tensor([score for _, score in data])
    return features, labels

In [34]:
processed_train_data = preprocess_imdb(train_data, vocab_imdb)

In [35]:
processed_test_data = preprocess_imdb(test_data, vocab_imdb)

In [36]:
processed_train_data[0].shape, processed_train_data[1].shape

(torch.Size([25000, 500]), torch.Size([25000]))

In [37]:
processed_test_data[0].shape, processed_test_data[1].shape

(torch.Size([25000, 500]), torch.Size([25000]))

### 10.7.1.3 创建数据迭代器

In [64]:
len(train_data), len(test_data)

(25000, 25000)

In [99]:
batch_size = 64

# train_set = Data.TensorDataset(*preprocess_imdb(train_data[:1024], vocab_imdb))
# test_set = Data.TensorDataset(*preprocess_imdb(test_data[:1024], vocab_imdb))

train_set = Data.TensorDataset(*preprocess_imdb(train_data, vocab_imdb))
test_set = Data.TensorDataset(*preprocess_imdb(test_data, vocab_imdb))

train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)

In [67]:
for X, y in train_iter:
    print('X', X.shape, 'y', y.shape)
    break
'#batches:', len(train_iter)

X torch.Size([64, 500]) y torch.Size([64])


('#batches:', 16)

In [68]:
X.T.shape, X.permute(1, 0).shape

(torch.Size([500, 64]), torch.Size([500, 64]))

## 10.7.2 使用循环神经网络的模型

In [69]:
embed_size, num_hiddens, num_layers = 100, 100, 2

In [70]:
embedding = nn.Embedding(len(vocab_imdb), embed_size)
embedding.weight.data.shape

torch.Size([37894, 100])

In [71]:
embeddings = embedding(X.permute(1,0))
# embeddings = embedding(X)
embeddings.shape

torch.Size([500, 64, 100])

In [72]:
encoder = nn.LSTM(
    input_size=embed_size, 
    hidden_size=num_hiddens, 
    num_layers=num_layers,
    bidirectional=True
)

In [73]:
outputs, _ = encoder(embeddings)

In [74]:
outputs.shape

torch.Size([500, 64, 200])

In [75]:
encoding = torch.cat((outputs[0], outputs[-1]), -1)
encoding.shape

torch.Size([64, 400])

In [76]:
decoder = nn.Linear(4*num_hiddens, 2)
outs = decoder(encoding)
outs.shape

torch.Size([64, 2])

In [100]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 2) # 初始时间步和最终时间步的隐藏状态作为全连接层输入

    def forward(self, inputs):
        # inputs的形状是(批量大小, 词数)，因为LSTM需要将序列长度(seq_len)作为第一维，所以将输入转置后
        # 再提取词特征，输出形状为(词数, 批量大小, 词向量维度)
        embeddings = self.embedding(inputs.permute(1, 0))
        # rnn.LSTM只传入输入embeddings，因此只返回最后一层的隐藏层在各时间步的隐藏状态。
        # outputs形状是(词数, 批量大小, 2 * 隐藏单元个数)
        outputs, _ = self.encoder(embeddings) # output, (h, c)
        # 连结初始时间步和最终时间步的隐藏状态作为全连接层输入。它的形状为
        # (批量大小, 4 * 隐藏单元个数)。
        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        outs = self.decoder(encoding)
        return outs

In [102]:
embed_size, num_hiddens, num_layers = 100, 100, 2
net = BiRNN(vocab_imdb, embed_size, num_hiddens, num_layers)
net

BiRNN(
  (embedding): Embedding(37894, 100)
  (encoder): LSTM(100, 100, num_layers=2, bidirectional=True)
  (decoder): Linear(in_features=400, out_features=2, bias=True)
)

In [103]:
net.zero_grad()

### 10.7.2.1 加载预训练的词向量

In [51]:
glove_vocab = Vocab.GloVe(name='6B', dim=100, cache=os.path.join(BASE, "glove"))

/content/glove/glove.6B.zip: 862MB [02:43, 5.28MB/s]                           
100%|█████████▉| 399999/400000 [00:22<00:00, 17883.32it/s]


In [55]:
# dir(glove_vocab)

In [56]:
glove_vocab.vectors.shape

torch.Size([400000, 100])

In [104]:
glove_vocab.vectors[0].shape[0]

100

In [79]:
def load_pretrained_embedding(words, pretrained_vocab):
    """从预训练好的vocab中提取出words对应的词向量"""
    embed = torch.zeros(len(words), pretrained_vocab.vectors[0].shape[0]) # 初始化为0
    oov_count = 0 # out of vocabulary
    for i, word in enumerate(words):
        try:
            idx = pretrained_vocab.stoi[word]
            embed[i, :] = pretrained_vocab.vectors[idx]
        except KeyError:
            oov_count += 1
    if oov_count > 0:
        print("There are %d oov words." % oov_count)
    return embed

In [105]:
net.embedding.weight.data.copy_(load_pretrained_embedding(vocab_imdb.get_itos(), glove_vocab))
net.embedding.weight.requires_grad = False # 直接加载预训练好的, 所以不需要更新它

There are 1335 oov words.


### 10.7.2.2 训练并评价模型

In [108]:
lr, num_epochs = 0.01, 5
# optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
optimizer = torch.optim.Adam([p for p in net.parameters() if p.requires_grad], lr=lr)
loss = nn.CrossEntropyLoss()

In [109]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

* `.cpu()` copies the tensor to the CPU, but if it is already on the CPU nothing changes.



In [110]:
def train(train_iter, test_iter, net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            # nn.utils.clip_grad_norm_(model.parameters(), 3)
            nn.utils.clip_grad_norm_(
                [p for p in net.parameters() if p.requires_grad], 
                3
            )
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(f'epoch {epoch + 1}, loss {train_l_sum:.3f}, train acc {train_acc_sum / n:.3f}, test acc {test_acc:.3f}, time {time.time() - start:.3f} sec')

In [111]:
train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

training on  cuda:0
epoch 1, loss 186.583, train acc 0.765, test acc 0.847, time 119.786 sec
epoch 2, loss 134.973, train acc 0.855, test acc 0.838, time 119.476 sec
epoch 3, loss 121.702, train acc 0.873, test acc 0.863, time 119.709 sec
epoch 4, loss 108.513, train acc 0.888, test acc 0.866, time 119.373 sec
epoch 5, loss 94.867, train acc 0.904, test acc 0.868, time 119.488 sec


In [149]:
# 本函数已保存在d2lzh包中方便以后使用
def predict_sentiment(net, vocab, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor([vocab[word] for word in sentence.lower().split()], device=device)
    label = torch.argmax(net(sentence.view((1, -1))), dim=1)
    return 'positive' if label.item() == 1 else 'negative'

In [143]:
device = list(net.parameters())[0].device
sentence = "this moovie is really so bad"
sentence = torch.tensor([vocab_imdb[word] for word in sentence.lower().split()], device=device)

In [144]:
sentence

tensor([   11, 25388,     6,    65,    37,    76], device='cuda:0')

In [145]:
net(sentence.view((1, -1)))

tensor([[ 0.3388, -0.2412]], device='cuda:0', grad_fn=<AddmmBackward>)

In [146]:
torch.sigmoid(net(sentence.view((1, -1)))).sum()

tensor(1.0239, device='cuda:0', grad_fn=<SumBackward0>)

In [151]:
# sentence = "this movie is so great"
sentence = "this movie is so bad"
predict_sentiment(net, vocab_imdb, sentence)

'negative'

In [153]:
# help(nn.utils.clip_grad_norm_)
sentence = "I don't like this movie a lot"
predict_sentiment(net, vocab_imdb, sentence)

'positive'

In [121]:
RegexpTokenizer(r'\w+').tokenize(sentence)

['I', 'don', 't', 'like', 'this', 'movie', 'very', 'much']

In [123]:
word_tokenize(sentence)

['I', 'do', "n't", 'like', 'this', 'movie', 'very', 'much']